# Cryptographic Integrity

The techniques described in the previous lecture are great when data are unintentionally corrupted. Next, we consider building tamper-resistant data structures.

## Cryptographic Hash Functions
A cryptographic hash function is a particularly strng hash function that is suitable for use in cryptography. Essentially, it is a hash function that is very hard to "reverse engineer" (look at a hash value and figure out an input that generates it). Like a regular hash function, it maps data of arbitrary size (often called the "message") to a bit string of a fixed size (the "hash value", "hash", or "message digest"). Ideally, the only way to find a message that produces a given hash is to attempt a brute-force search of possible inputs to see if they produce a match. The reason that we don't always use cryptographic hashes is that they are expensive to compute and if you have no reason to defend against intentional manipulation, they are wasteful to use. Python has libraries for computing such hashes:

In [5]:
import hashlib
m = hashlib.sha256()
m.update(b"Nobody inspects")
print('Nobody inspects =>', m.hexdigest())

m.update(b"the bear")
print('the bear =>', m.hexdigest())

Nobody inspects => e7a3f808cb0687fd3660e956a5df0f00e23edac5650769ec354ee670b658858c
the bear => 12f9f73cf1dff081f84acc6a4e3bbb7e492f31f2ae8136f58d4204eb7e92ab0b


## Hash Lists
Let's consider the following example where we want to hash a relatively long string:

In [6]:
m.update(b"the quick brown fox jumped over the lazy dog")
print(m.hexdigest())

574dfb2660b156fb9c6128264cca0a94e8b5e5e973b845ac459a9703251d14ee


Suppose one of those characters was tampered with (an extra space after quick):

In [9]:
m.update(b"the quick  brown fox jumped over the lazy dog")
print(m.hexdigest())

01b17f5a3ab28c81c42ed55239d7ea940eab01d9274edf3babb474b8727081fa


The change in hash value only tells you that there is some modification you have no idea where that modification is or how significant it is. It could be minor (like a change in the example above where there is a single extra space) or it could be major like the entire string is modified. The nature of a cryptographic hash makes it difficult to differentiate between such cases. 

One solution is to break the data into smaller blocks and hash each of the blocks individually. 

In [16]:
def hashlist(data, blocksize=4):
    datablocks = []
    
    for i in range(blocksize, len(data), blocksize):
        m.update(bytes(data[i-blocksize:i]))
        datablocks.append((i, m.hexdigest()))
    
    #deal with the last block
    if i != len(data) - 1:
        m.update(bytes(data[i:len(data)]))
        datablocks.append((len(data), m.hexdigest()))
        
    return datablocks

hashlist(b"the quick  brown fox jumped over the lazy dog", blocksize=12)

[(12, '51c9c458bad4def01e660dbe88b543ffb4f22d2e2862eadc06f113fa8fb33651'),
 (24, '45e8f808b2b813ec5cbb106b9b1a92280540b589bcc0cd7e5f3e98e93f489172'),
 (36, '43c8cec4930559f2af27abbf7968f6fa721613a7fa9662f299c64233f79e7c93'),
 (45, '32b9f1215bc9b16eef72c08a306c4773cf15e8b36f5d512b4699ca2100b8e8f0')]

Then, we can check each block individually to see if it has been tampered with. The downside is that determining whether there is any data modification can be expensive if the size of the data set is very large (have to check each block). 

## Merkle Tree

But, we run into the same problem as the CRC, what if the signature itself is tampered with? Such a problem can happen in a decentralized setting where each node in a network has its own data and hosts its own signature. The solution here is simple, we simply add a hash signature to the hashes that have been generated. These signatures of signatures can be copied and placed on multiple different nodes to decentralize the trust.

In [2]:
blocks = hashlist(b"the quick  brown fox jumped over the lazy dog", blocksize=12)
print('Blocks', blocks)

print()

m.update(bytes(str(blocks),encoding='utf8'))
print('All Blocks', m.hexdigest())

NameError: name 'hashlist' is not defined

If we had a single signature-of-signatures, there is a subtle issue that a failure means that at least one of the consituent data failed a check for manipulation (but you don't know which one!). A smarter approach is to recursively build a tree data structure, where every parent hashes a list of its children's hashes. To identify tampered blocks, you simply traverse down the tree identifying mismatched hashes. The tree takes the form as follows (from wikipedia):

![Merkle Tree](https://upload.wikimedia.org/wikipedia/commons/thumb/9/95/Hash_Tree.svg/1024px-Hash_Tree.svg.png)

This basic insight is the data structure behind the blockchain and many cryptographic currencies. By design, a such a data structure is resistant to modification of the data. Thus it is perfectly suited to manager "an open, distributed ledger that can record transactions between two parties efficiently and in a verifiable and permanent way". For use as a distributed ledger, a the data structure would have to be managed by a peer-to-peer network where signatures are replicated. Once recorded, the data in any given block cannot be altered retroactively without alteration of all subsequent blocks, which requires consensus of the network majority. Although blockchain records are not unalterable, blockchains may be considered secure by design and exemplify a distributed computing system with high Byzantine fault tolerance.